# Tutorial Session 2
Leader: Oliver Beckstein

Note: This is the **instructor copy**, for the tutorial use the **participant copy** (which is based on this copy with most of the code removed).

## Outline

￼1. trajectory processing (reading)
   - random access
   - dynamically updated trajectory data (AtomGroup.positions)
   - Examples
      - get CV timeseries and plot (AdK angles from DIMS)
      - Ramachandran analysis of trajectory (2D histogram)
      - MAYBE: velocity autocorrelation function of e.g. ions or water with scipy.fft and diffusion coefficient
      - segment trajectory based on order parameter (e.g. detect conformational transition in equilibrium AdK trajectory)
￼   - modifying coordinates
      -translation and rotation
      - RMSD superposition
         - Example: AdK domains

￼2. trajectory processing (writing)
   - writing coordinates (Writer)
   - Example: format changes (Multi PDB/XTC) and concatenation (ChainReader)

3. Bonus: interfacing with other packages
     - Examples:
        - implement LeafletFinder with networkx and analyze bilayer formation in a CGSA-MD simulation (@orbeckst)
        - pandas with MDAnalysis (@dotsdl)

        
       

## Data files
Defined `DATADIR` to point to the directory where you [downloaded the tutorial trajectories](http://becksteinlab.github.io/MDAnalysis-workshop/datadownload.html) to.

In [4]:
import os.path
DATADIR = os.path.expanduser("~/Workshops/CECAM/tutorial/sandbox")

In [5]:
DATADIR

'/home/oliver/Workshops/CECAM/tutorial/sandbox'

## Package imports 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
import MDAnalysis as mda
print(mda.__version__)

0.12.1


## Analyzing trajectories
Reading data from a trajectory, frame by frame:

## Writing trajectories and selections
Writing out coordinates; introduce the selection writer (cross package).

## Extra 